In [ ]:
import json
import datetime
import os
import time
import sys

import pandas as pd
import numpy as np
from sklearn import preprocessing

import matplotlib.pyplot as plt

sys.path.append('..')
from preprocessing import ohe_explicit

In [ ]:
def load(path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'},
                     nrows=nrows)
    
    # Normalize JSON columns
    for column in JSON_COLUMNS:
        column_as_df = pd.io.json.json_normalize(df[column])
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    
    # Parse date
    df['date'] = df['date'].apply(lambda x: pd.datetime.strptime(str(x), '%Y%m%d'))
    print("Loaded file {}\nShape is: {}".format(path, df.shape))
    return df

def process(train, test):
    print("Dropping constant columns...")
    
    # Remove columns with constant values.
    const_cols = [c for c in train.columns if train[c].nunique(dropna=False) == 1]
    train = train.drop(const_cols, axis=1)
    test = test.drop(const_cols, axis=1)
    
    # Cast target
    train["transactionRevenue"] = train["transactionRevenue"].fillna(0).astype(float)
    train["target"] = np.log(train["transactionRevenue"] + 1)
    del train["transactionRevenue"]
    
    train_len = train.shape[0]
    merged = pd.concat([train, test], sort=False)

    # Change values as “not available in demo dataset”, “(not set)”, “unknown.unknown”, “(not provided)” to nan\n",
    list_missing=["not available in demo dataset", "(not provided)", "(not set)", "<NA>", "unknown.unknown",  "(none)"]
    merged=merged.replace(list_missing, np.nan)

    # Create some features.
    merged['diff_visitId_time'] = merged['visitId'] - merged['visitStartTime']
    merged['diff_visitId_time'] = (merged['diff_visitId_time'] != 0).astype(int)
    del merged['visitId']
    del merged['sessionId']

    print("Generating date columns...")
    merged['WoY'] = merged['date'].apply(lambda x: x.isocalendar()[1])
    merged['month'] = merged['date'].apply(lambda x: x.month)
    merged['quarterMonth'] = merged['date'].apply(lambda x: x.day // 8)
    merged['weekday'] = merged['date'].apply(lambda x: x.weekday())
    del merged['date']

    format_time = lambda t: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(t))
    merged['visitHour'] = pd.to_datetime(merged['visitStartTime'].apply(format_time)).apply(lambda t: t.hour)
    del merged['visitStartTime']
    
    
    print("Finding total visits...")
    # This could be considered an information leak as I am including information about the future when predicting
    # the revenue of a transaction. In reality, when looking at the 3rd visit we would have no way of knowning
    # that the user will actually shop X more times (or if he will visit again at all). However since this
    # info also exists in the test set we might use it.
    total_visits = merged[["fullVisitorId", "visitNumber"]].groupby("fullVisitorId", as_index=False).max()
    total_visits.rename(columns={"visitNumber": "totalVisits"}, inplace=True)
    merged = merged.merge(total_visits)

    print("Splitting back...")
    train = merged[:train_len]
    test = merged[train_len:]
    return train, test

def preprocess_and_save(data_dir):
    train = load(os.path.join(data_dir, "train.csv"))
    test = load(os.path.join(data_dir, "test.csv"))

    target = train['transactionRevenue'].fillna(0).astype(float)
    train['target'] = target.apply(lambda x: np.log1p(x))
    del train['transactionRevenue']

    train, test = process(train, test)
    train.to_csv(os.path.join(data_dir, "preprocessed_train.csv"), index=False)
    test.to_csv(os.path.join(data_dir, "preprocessed_test.csv"), index=False)    
    

# Call this to save the preprocessed data for later use
# preprocess_and_save("../data/")

In [ ]:
train = load("../data/train.csv", nrows=100000)
test = load("../data/test.csv", nrows=10000)

train, test = process(train, test)

# Categoricals with many values

There are a number of categorical features with many different values. This is an issue, specifically in the cases where those categorical features are not ordinal, and therefore label encoding them does not make sense. The only choice we are left with, is OHE. However naively performing this step would add hundreds of columns for each original categorical feature - in the end yielding potentially thousands of super sparse features. What I would like to explore, is whether there exist **specific values** with predictive value significantly higher than average. For example, looking at the particular country might be a weak predictor. However there might be 5 specific countries with a huge revenue deviation from the average (and enough samples to consider this discrepancy statistically significant).

In [ ]:
countries = train['country']
print("There are {} different countries in our dataset".format(len(countries.unique())))

In [ ]:
aggregations = {'target':['mean', 'count']}

countries = train[["country", "target"]].groupby("country", as_index=False).agg(aggregations)
countries.columns = ["country", "targetMean", "occurenceCount"]

# Let's focus only on countries with multiple records to preserve some statistical significance
keep = 10
usual_countries = countries.sort_values("occurenceCount", ascending=False).head(keep)

global_average = train["target"].mean()
usual_countries["deviation"] = usual_countries["targetMean"] - global_average
usual_countries.plot.bar(x="country", y="deviation")

# USA seems quite different

What we can find here is that USA is very different to any other country with significant sample count. What we can do with this information? We instead of using OHE to code every country in our dataset, we can probably get away with a single boolean column: **is this record coming from the USA?**

In [ ]:
cities = train['city']
print("There are {} different cities in our dataset".format(len(cities.unique())))

In [ ]:
aggregations = {'target':['mean', 'count']}

cities = train[["city", "target"]].groupby("city", as_index=False).agg(aggregations)
cities.columns = ["city", "targetMean", "occurenceCount"]

# Let's focus only on countries with multiple records to preserve some statistical significance
keep = 10
usual_cities = cities.sort_values("occurenceCount", ascending=False).head(keep)

global_average = train["target"].mean()
usual_cities["deviation"] = usual_cities["targetMean"] - global_average
usual_cities.plot.bar(x="city", y="deviation")

# What about the cities?

Here we can see some pretty strong deviations. However we need to note that the top ones come from US cities, so part of the variance these cities explain, is already included in the information that they belong to the USA. However their deviation is considerably higher than that of USA alone (1.0 vs 0.3) so including those columns might still be beneficial. The deviation we see is actually very distorted because of the USA outlier. Perhaps it would make more sense to only focus on the deviation from the average of non-USA cities.

In [ ]:
train_not_us = train[train['country'] != "United States"]
train_us = train[train['country'] == "United States"]

outside_us_avg = train_not_us['target'].mean()
us_avg = train_us["target"].mean()

print("Average in US: {}\nAverage outside US: {}".format(us_avg, outside_us_avg))

**Let's then repeat out analysis but this time separatly for the pieces of data**

In [ ]:
aggregations = {'target':['mean', 'count']}

# US case
def city_deviation(df, title="Deviation per city"):
    cities = df[["city", "target"]].groupby("city", as_index=False).agg(aggregations)
    cities.columns = ["city", "targetMean", "occurenceCount"]

    # Let's focus only on countries with multiple records to preserve some statistical significance
    keep = 10
    usual_cities = cities.sort_values("occurenceCount", ascending=False).head(keep)

    global_average = df["target"].mean()
    usual_cities["deviation"] = usual_cities["targetMean"] - global_average
    ax = usual_cities.plot.bar(x="city", y="deviation", title=title, rot=45, legend=False)
    ax.set_xlabel("City")
    ax.set_ylabel("Deviation")
    
city_deviation(train_us, title="Deviation from the mean - US")
city_deviation(train_not_us, title="Deviation from the mean - Rest of the world")

## Much better!

Now we can clearly see what information should be included besides the country (or to be exact, whether or not the country is the US). For example it makes no sense to include Los Angeles or Mountain View even though they deviate from the global average, because all this deviation is explained by the fact that they exist in the US! Instead we should include Chicago, New York, Austin, Seattle and maybe Palo Alto. And as we can see the deviations are much smaller outside the US, with the exception of Toronto which MUST be included.

### Food for thought
It makes sense that deviations outside the US are smaller because the target itself is considerable lower. Perhaps we should look at relative deviations instead?

In [ ]:
check = ohe_explicit(train)
check.head()  